![title](https://ucm.es/themes/ucm16/media/img/logo.png)

# Bases de datos NoSQL

### Práctica Redis. Servicios de publicación/subscripción 

#### Parte 1. Publicación

Vamos a publicar las noticias de una página web- Necesitaremos las bibliotecas:

* requests: para bajar páginas completas a partir de la url
* bs4: beatifulsoap, para procesar los htmls buscando las cabeceras

Si no tenemos instala alguna de ellas, desde una consola de Anaconda (Anaconda prompt)

    pip install bs4
    pip install requests

Lo primero,  el nombre:

Nombre: Arturo Barbero Pérez

Ahora conectamos a Redis. Es importante que cambieis los datos por los de vuestra cuenta, para no saturar el servidor

In [1]:
# Ojo, cambiar estos datos por los de vuestro acceso a red en redislabs
redisconexion = "redis-12588.c135.eu-central-1-1.ec2.cloud.redislabs.com"
redispuerto = 12588
passwd = "YNUBRlaFh6kJnTKfDxsXur44M3jOkNqy"

import redis
r = redis.Redis(host=redisconexion, password=passwd, port=redispuerto, charset="utf-8", decode_responses=True, db=0)
r.ping() # debe mostrar True

True

### Pregunta 1: 4 puntos

La siguiente función es la que tenéis que modificar. Le van a ir llegando titulares, que son strings. Ahora mismo solo los muestra por pantalla, pero hay que reemplazar el print por vuestro código.

En particular:

* Si el titular contiene al menos un símbolo monetario ('₹','$','£','€') seguidos por un dígito dígito 0,1...9, debe enviarse por el canal `news-money`

* Si no se da el caso anterior, pero el titular contiene alguna de las palabras del array "especiales" enviará la noticia por el canal `news-selected`.  Nota: basta con que tenga la secuencia de caracteres consecutivos, no hace falta mirar si tiene espacio antes o después, etc. Por ejemplo la palabra 'abra' estaría según estadefinición dentro de la frase "y dijo ¡abracadabra!"

* En otro caso enviará  por `news-other`

Completar las funciones  `contiene_monetario`, `contiene_selected`y utilizarlas en trata_titular para publicar cada titular en el canal adecuado 


In [91]:
# Utilizo expresiones regulares para parsear el string
import re

# La expresión tiene que tener la forma: (digit + al menos un simbolo), o lo contrario. 
def contiene_monetario(titular, simbolos = ['₹','$','£','€']):
    symbols = "".join(simbolos)
    expression = "\d+[" + symbols + "]|[" + symbols +"]\d+"                                      
    foundResults = re.findall(expression, titular)
    if(foundResults):
        return True
    else:
        return False
    
    
# comprueba si 'titular' tiene una palabra especial
# devuelve True en el primer caso o False en otro caso
def contiene_selected(titular, especiales = ["US", "India", "Spain", "UK", "Trump"]):
    specials = "|".join(especiales)
    foundResults = re.findall(specials, titular)
    if foundResults:
        return True
    else:
        return False
    

# publica el titular o bien en 'news-money', o en 'news-selected' o en 'news-other'        
def trata_titular(titular):
    if contiene_monetario(titular):
        r.publish("news-money", titular)
    elif contiene_selected(titular):
        r.publish("news-selected", titular)
    else:
        r.publish("news-other", titular)

Se puede usar este código para probar `contiene_monetario` y `contiene_selected`. Debe mostrar


    False
    True
    True
    True
    False

In [92]:
# debe devolver false
print(contiene_monetario("Railways announces 2.3 lakh vacancies; 10% EWS quota applicable"))
# debe devolver true
print(contiene_monetario("Zynga Co-founder raising $700 mn for investment fund"))
# debe devolver true
print(contiene_selected("Self-driving robots to deliver food on US university campus"))
# debe devolver true
print(contiene_selected("Wonderful work of Indian cinema: Sadhguru on 'Manikarnika'"))
# debe devolver false
print(contiene_selected("Your sense of sanctity does not lie in your body: Swara Bhasker"))


False
True
True
True
False


### Publisher

La siguiente casilla de código es la que debemos ejecutar antes de hacer cualquier prueba tras una subscripción, para alimentar al subscriptor de mensajes. No hay que tocar ni cambiar nada.

El código baja titulares  de forma continua hasta alcanzar las 100 noticias.  Para cada titular llama a la función anterior `trata_titular`. Lo he sacado de este sitio:

https://www.pythoncircle.com/post/678/python-script-14-scraping-news-headlines-using-python-beautifulsoup/


In [96]:
import requests
from bs4 import BeautifulSoup
import json

MAX_NEWS = 100


def headlines(response_text):
    total = 0
    soup = BeautifulSoup(response_text, 'lxml')
    headlines = soup.find_all(attrs={"itemprop": "headline"})
    for headline in headlines:
        trata_titular(headline.text)
        total+=1
    return total


def get_headers():
    return {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-IN,en-US;q=0.9,en;q=0.8",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "cookie": "_ga=GA1.2.474379061.1548476083; _gid=GA1.2.251903072.1548476083; __gads=ID=17fd29a6d34048fc:T=1548476085:S=ALNI_MaRiLYBFlMfKNMAtiW0J3b_o0XGxw",
        "origin": "https://inshorts.com",
        "referer": "https://inshorts.com/en/read/",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        "x-requested-with": "XMLHttpRequest"
    }


url = 'https://inshorts.com/en/read'
response = requests.get(url)
total = headlines(response.text)

# get more news
url = 'https://inshorts.com/en/ajax/more_news'
news_offset = "apwuhnrm-1"

while total<MAX_NEWS:
    response = requests.post(url, data={"category": "", "news_offset": news_offset}, headers=get_headers())
    if response.status_code != 200:
        print(response.status_code)
        break

    response_json = json.loads(response.text)
    total += headlines(response_json["html"])
    news_offset = response_json["min_news_id"]